In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')

In [2]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))  

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]    
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))    

In [3]:
# parameters
customer_id = '5'
formatted_attribute = 'Material'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Window Blinds"""
buckets = """Package Quantity"""

regex_pattern=r'(?i)(x\s?\d+.?\d*\".?D)|(.?.?.?.?.?Depth.?.?.?.?)()'

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
curated_by=762

In [4]:
# customer_id = '5'
# customer_name='%bedbathandbeyond%'
# buckets = """Multiple Buckets"""
# strategy_version_input=677
# attribute_id_input=426
# formatted_attribute = 'Material'

In [5]:
# # parameters
# attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
# value='%n/a%'
# params = {'customer_id': customer_id ,
#           'attribute': attribute,
#           'buckets': str(buckets.split('\t'))[1:-1],
#           'value':value,
#           'customer_name':customer_name
#          }
# curation_col = f'Q:{attribute}'

In [6]:
# buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
# INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
# WHERE sba.attribute_id = {attribute_id_input}
# AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]

In [7]:
# df = query_from_file(file_name='./query/uncurated.sql', params=params)
# print(len(df))
# df = df[df['bucket_id'].isin(buckets)]
# len(df)

In [8]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
# list_values

In [43]:
# df = query_from_file(file_name='Curate_wipe/BusinessIntelligence/Platform Queries/Bucket Attribute Values for All active Strategies.sql', params=params)
df = query_from_file(file_name='query/Delete_Motion_Max_Min_temp_date.sql', params=params)
# df=df[df['value']=='n/a']
print(len(df))
df['buckets'].explode().value_counts()[0:20]

414788


Rugs & Mats                           68763
Curtains                              31867
Rugs & Mats B                         31466
Bedding Variety Packs                 24630
Prints, Posters, & Paintings          19451
Throw/Decorative Pillows              12093
Tablecloths & Table Runners           11584
Beds/Bedsteads/Bedframes/Bases         8380
Vacuum/Water Bottles & Travel Mugs     7990
Blankets/Throws                        6831
Window Blinds                          6287
Glasses                                5966
Lamps                                  5231
Chairs                                 5032
Pillowcases & Pillow Protectors        5030
Mugs/Teacups/Cups                      4228
Plates & Platters                      3854
Outdoor Games & Activities             3849
Hanging Accents & Accessories          3783
Stools                                 3624
Name: buckets, dtype: int64

In [11]:
err

NameError: name 'err' is not defined

In [ ]:
# df[df['buckets'].astype(str)=='Duvet Covers']['value'].explode().value_counts()

In [ ]:
## df['buckets'].explode().value_counts()
#   Hand Towels
# Kitchen/Dishtowels & Dishcloths

In [44]:
regex_pattern = r"(?i)(resin)|(rubber)|(polyester)|()" 
resin=df
resin['matches'] =resin['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

In [45]:
''''
Buckets for Category 13268:
-----Do we need to change the guidelines to say that cushions are not counted as Polyester
Chairs--Keep Polyester
Benches/Storage Benches--Keep Polyester--Removable Cushions
Temporary Seating--Keep Polyester
Ottomans/Poufs/Cubes/Footrests--Keep Polyester
Stools--Keep Polyester--uncurate Cushion
Rockers/Gliders--Keep Polyester
Chair Swings------------------------------------------------------------Remove polyester--Example? -------------------------------------------------------------------68206352
Furniture Variety Packs--Keep Polyester but curate
Chaises & Loungers--Keep Polyester but curate for cushions
Chair Accessories & Replacement Parts--Keep Polyester--100%
Papasan Chairs--Keep Polyester but curate for cushions
Shoe Racks & Holders/Organizers--Keep Polyester 100%
Seating Cushions--Keep Polyester
'''

In [82]:
regex_pattern = r"(?i)(polyester)|()" 
chair=resin[resin['buckets'].astype(str)=='Chairs']
chair['matches'] =chair['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(chair[chair['matches'].astype(str)!='[]']))
chair[chair['matches'].astype(str)!='[]']

1235


attributes snake_case_name  \
572      Material        material   
7423     Material        material   
9598     Material        material   
10121    Material        material   
10967    Material        material   
...           ...             ...   
269420   Material        material   
269529   Material        material   
269530   Material        material   
269531   Material        material   
269532   Material        material   

                                                                        name  \
572     Baxton Studio Sabella 7-Piece Reclining Sofa in Dark Grey/Light Grey   
7423                TOV Furniture Gardenia Velvet Dining Chair in Light Grey   
9598                INK+IVY® Boomerang Side Dining Chairs in Blue (Set of 2)   
10121                                TOV Furniture Eva Velvet Chair in Blush   
10967                          Ink+ivy™ Upholstered Dining Chairs (Set of 2)   
...                                                                      ...   
269420             Polyester Upholstered Faux Fur Saucer Chair Chair in Pink   
269529                         Urban Shop Faux Fur Upholstered Chair in Grey   
269530                        Urban Shop Faux Fur Upholstered Chair in Taupe   
269531                           Urban Shop Sherpa Flip Lounge Chair in Grey   
269532                          Urban Shop Sherpa Flip Lounge Chair in Taupe   

       external_id  \
572       64306698   
7423      69610381   
9598      47578593   
10121     63772784   
10967     64897622   
...            ...   
269420    67102440   
269529    65449899   
269530    65449875   
269531    65449868   
269532    65449844   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      long_desc  \
572     With room for the whole family, the 7-piece, 2-tone Sabella Reclining Sofa from Baxton Studio brings both plush comfort and warm style to your home. Features generously padded left-facing and right-facing recliners that are easy to operate. Brings incredible comfort and contemporary style to your home 7-piece set includes: 1 right arm facing recliner 1 left arm facing recliner 2 armless chairs 1 corner unit 2 consoles Pull handle control recliner Each piece connects to each other with built-in hinges 2-tone design Pillowtop armrests Pipe detail 2 cup holders Wood/metal/foam/polyester/metal/plastic 100% foam fill Spot clean Assembly required Measures 100\" D x 110.24\" W x 40.16\" H 18.9\" seat height Weighs 361.56 lb. 900 lb. weight capacity 30-day manufacturer's warranty Imported   
7423                                                                                   Handmade of solid pine by skilled furniture craftsmen, the Gardenia Velvet Dining Chair from TOV Furniture brings lavish style and comfort to your dining room. Features soft velvety upholstery with scallop tufting to enhance any dining area. Adds a touch of glamorous style to any dining area Handmade by skilled furniture craftsmen Sturdy and durable solid pine wood frame Buttery soft polyester velvet upholstery Scallop tufted design High-density foam fill for ultimate comfort Solid pine wood/metal/foam/polyester velvet 100% foam fill Spot clean Easy to assemble Measures 21.9\" L x 22.6\" W x 31.5\" H 19.3\" seat height Weighs 17 lb. 27

In [ ]:
'''
Chaises & Loungers--Keep Polyester but curate for cushions
Seating Cushions--Keep Polyester
Covers/Protectors--Keep Polyest 100%
Sectionals--Keep ployester 100%
Throw/Decorative Pillows--Keep Polyester 100%
Furniture Variety Packs--Keep Polyester but curate
Daybeds-----------------------------------------------------------------Ask Ross if they mean the actual bed
Chair Swings------------------------------------------------------------Remove polyester--Example? -------------------------------------------------------------------68206352
Ottomans/Poufs/Cubes/Footrests--Keep Polyester
Blankets/Throws---------------------------------------- not including the fringe tie 100% polyester
Sleeper Sofas/Sofa Beds/Futons--Keep Polyester
Chairs--Keep Polyester
Pool Recreation Accessories----------------Mesh and polyester cover 
'''

In [79]:
regex_pattern = r"(?i)(polyester)|()" 
chair=resin[resin['buckets'].astype(str)=='Pool Recreation Accessories']
chair['matches'] =chair['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(chair[chair['matches'].astype(str)!='[]']))
chair[chair['matches'].astype(str)!='[]'][0:500]

2


attributes snake_case_name  \
151620   Material        material   
164222   Material        material   

                                                       name external_id  \
151620  SwimWays® Spring Float Sit-Up Pool Recliner in Blue    13789924   
164222           SwimWays Fold Up Spring Pool Float in Blue    12059353   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           long_desc  \
151620  Soak up the sun in style with the SwimWays Spring Float Pool Recliner. This sit-up float includes a backrest, headrest, cup holder, and ottoman for a fully-prepared paradise, providing rest and relaxation right in your own backyard. Drift along with the pool waves as you recline in this float spring pool float that has all you need for relaxing poolside in style Recline floating along the water with included relaxation tools to spend all day by your pool with ease Recommended for ages 13 and up 250 lbs. maximum weight capacity Jet Valve technology allows for fast inflation and deflation Secure handles for stable relaxation Mesh and polyester cover the spring float recliner for a comfortable surface to float in all day long Measures 69\" L x 35\" W x 5.5\" H when fully inflated Imported   
164222                                                                                                                                                                                                                                                           This inflatable Fold Up SwimWays Spring Pool Float makes pool time casual time. Unfold it, inflate, and relax your cares away as you float in your swimming pool. Convenient jet valve allows for quick inflating and deflating. Fantastic float makes any time at the pool more relaxing and enjoyable Inner spring design provides stability Headrest provides optimal comfort Folds up for compact storage 2-tone blue styling Includes a convenient carry case Suitable for ages 15 and up Measures 69\" L x 35\" W x 5.5\" H PVC/polyester Wipe clean Imported   

       curated_date  customers_id                      buckets  \
151620   2021-04-26             5  Pool Recreation Accessories   
164222   2021-04-24             5  Pool Recreation Accessories   

                   value   family_id resolution      matches  
151620            Fabric     5469139   standard  [polyester]  
164222  ["Fabric","PVC"]  1012059353   standard  [polyester]

In [ ]:
Furniture Variety Packs
Wall Decals & Appliques
Seating Cushions
Throw/Decorative Pillows
Dining Tables
Rugs & Mats
Chairs
Stools
Rugs & Mats B
Plant Holders & Accessories
Sconces & Vanity Lighting
Side/End Tables
Ottomans/Poufs/Cubes/Footrests
Covers/Protectors
Lanterns & Candle Holders
Office Chairs
Sofas/Couches/Loveseats/Settees
Beverage & Bottle Carriers
Coffee Tables
Benches/Storage Benches
Desks/Workstations/Workbenches
Curtains
Pet Travel Housing
Folding Tables & Tray Tables
Cabinets
Coolers & Food Carriers
Trees, Shrubs, Topiaries, & Other Plants
Boxes, Baskets, & Bins
Umbrellas/Parasols
Outdoor Sculptures, Decorations, & Figurines
Cribs/Crib Beds
Drawers
Vacuum Cleaners
Chairs
Desks/Workstations/Workbenches
Console/Entryway Tables
Activity Tables
Shelves & Shelving Units
High Chairs & Feeding, Booster, & Activity Seats
Pet Beds & Cots
Bookcases/Bookshelves
3
6767:Wall Mirrors
2
7092:Pet Houses, Crates, & Kennels
2
7095:Small Animal Housing
2
7134:Pet Toys
2
7138:Training Ramps & Steps
2
7355:Heating Equipment
2
7425:Chaises & Loungers
2
7459:Daybeds
2
7479:TV Stands/Media Consoles
2
7918:Pendant Lighting
2
8751:Desiccants
2
9092:Outdoor Grills/Barbecues & Smokers
2
6629:Play Pens, Safety Gates, & Enclosures
1
6768:Motion Sensors
1
6804:Weathervanes
1
7402:Candles
1
7421:Sectionals
1
7433:Chair Swings
1
7455:Beds/Bedsteads/Bedframes/Bases
1
7461:Buffets/Sideboards & Bar Counters
1
7482:Shoe Racks & Holders/Organizers
1
7485:Hall Trees
1
7496:Blankets/Throws
1
7913:Rope/String Lights
1
7920:Bowl & Vase Fillers
1
7935:Vases
1
7936:Hanging Accents & Accessories
1
8130:Prints, Posters, & Paintings
1
8141:Flush & Semi-Flush Mount Lighting
1
8166:Night Lights & Under Cabinet Lights
1
8418:Carpet/Steam Cleaners
1
8742:Step Stools
1
9150:Outdoor Fountains & Water Features
1
9345:Post Lighting
1
9591:Stands, Cradles, & Docks
1

In [47]:
resin[resin['external_id'].astype(str)=='60716330']

attributes snake_case_name  \
138455   Material        material   

                                                           name external_id  \
138455  Modway Encounter Patio Stand-Alone Swing Chair in White    60716330   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         long_desc  \
138455  A fun addition to your porch or patio, the Modway Encounter Patio Stand-Alone Swing Chair features wicker rattan style and great comfort. With sturdy construction, this wonderful swinging lounge chair stands on its own and doesn't require mounting. Modway Encounter Patio Stand-Alone Swing Chair is a marvelous addition to your backyard or porch Sturdy aluminum frame with all-weather material, a hanging chain apparatus, and plush all-weather fabric cushion UV- and weather-resistant fabric cushion Aluminum/synthetic wicker rattan/polyester Supports up to 265 lb. Measures 40.5\" L x 47\" D x 76.5\" H Some assembly required 1-year manufacturer's warranty Imported   

       curated_date  customers_id       buckets                   value  \
138455   2021-04-30             5  Chair Swings  ["Polyester","Wicker"]   

         family_id resolution      matches  
138455  1060716330   standard  [polyester]

In [48]:
resin[resin['external_id'].astype(str)=='65327050']

attributes snake_case_name                                        name  \
153167   Material        material  Keter Bahamas All-Weather Armchair in Grey   

       external_id  \
153167    65327050   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       long_desc  \
153167  The Keter Bahamas Armchair is union of superior comfort and design to give you the perfect all-weather outdoor armchair. Durable, weather-resistant rattan-style construction ensures this chair will never rust or fade, so you can relax worry-free. A durable, all-weather armchair with an attractive rattan texture for the ultimate summertime relaxation experience Coordinate with the Bahamas loveseat and coffee table (sold separately) Includes: Bahamas armchair Cushion Comfortable padded seat cushion and modular design with soft armrests Lightweight design makes these pieces easy to move and store Hardy weather-resistance and UV protection ensures this chair will never rust, rot, unravel, or fade Available in Brown and Grey colors Easy click-together assembly; no tools required Measures 31.1'' H x 27.6'' W x 29.5'' D   

       curated_date  customers_id buckets value family_id resolution matches  
153167   2021-04-25             5  Chairs   n/a   5194114   standard      []

In [49]:
resin[resin['external_id'].astype(str)=='62949422']

attributes snake_case_name  \
167035   Material        material   

                                                                           name  \
167035  Modway Sojourn Outdoor Chaise Lounge in Tuscan Sunbrella® Canvas Fabric   

       external_id  \
167035    62949422   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        long_desc  \
167035  Enhance your outdoor space with the Modway Sojourn Outdoor Chaise Lounge. Perfect on the patio or poolside, this elegant piece features durable Sunbrella cushions, a synthetic rattan weave, UV protection, and a powder-coated aluminum frame. Waterproof and UV-resistant wicker rattan with stylish stainless steel legs Synthetic rattan weave Plush all-weather cushions in durable Sunbrella® fabric Sunbrella fabric is engineered to endure the weather and resist fading, mildew, and stains Powder-coated aluminum frame Easy to assemble Aluminum/stainless steel/synthetic wicker/polyester Supports up to 882 lb. Measures 82.5\" L x 30\" W x 42\" H Weighs 52 lb. 1-year manufacturer's warranty Imported   

       curated_date  customers_id             buckets  \
167035   2021-04-22             5  Chaises & Loungers   

                                                      value   family_id  \
167035  ["Aluminum","Polyester","Stainless Steel","Wicker"]  1062949422   

       resolution      matches  
167035   standard  [polyester]

In [50]:
resin[resin['external_id'].astype(str)=='69605478']

attributes snake_case_name  \
165734   Material        material   

                                                                   name  \
165734  Bee & Willow™ Home Elmridge Wicker Folding Chair in Brown/Black   

       external_id  \
165734    69605478   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         long_desc  \
165734  Add a sleek and comfortable seat to your outdoor space with the Bee & Willow Home Elmridge Wicker Folding Chair. Suitable to place on your patio, this folding chair features hand-woven all-weather wicker with a steel frame. Suitable to place on your patio, this egg chair features hand-woven all-weather wicker with a steel frame Steel frame Assembly required Measures 19.88\" L x 18.11\" W x 32.67\" H 225 lb. weight capacity 1-year limited manufacturer's warranty Imported   

       curated_date  customers_id            buckets  \
165734   2021-04-22             5  Temporary Seating   

                                 value family_id resolution matches  
165734  ["Polyester","Steel","Wicker"]   5524119   standard      []

In [ ]:
rugs_mats=df[df['buckets'].astype(str)=='Rugs & Mats'][df['value'].astype(str)!='n/a']
regex_pattern = r"(?i)(leather)|()" 
rugs_mats['matches'] =rugs_mats['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
rugs_mats_df=rugs_mats[rugs_mats['matches'].astype(str)!='[]']
rugs_mats_df

In [ ]:
curtains=df[df['buckets'].astype(str)=='Curtains'][df['value'].astype(str)!='n/a']
# regex_pattern = r"(?i)(leather)|()" 
# rugs_mats['matches'] =rugs_mats['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# rugs_mats_df=rugs_mats[rugs_mats['matches'].astype(str)!='[]']
curtains['value'].explode().value_counts()

In [ ]:
df[df['buckets'].astype(str)=='Curtains'][df['value'].astype(str)!='n/a']['value'].explode().value_counts()

In [ ]:
regex_pattern = r"(?i)(Resin)|()" 
df['matches'] =df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
final_df=df[df['matches'].astype(str)!='[]']
final_df

In [ ]:
regex_pattern = r"(?i)(Acrylic)|()" 
df['matches'] =df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
final_df=df[df['matches'].astype(str)!='[]']
final_df

# Seperate Bukets and start curating

# Wall Sculptures

In [ ]:
wall=df[df['buckets'].astype(str)=='Wall Sculptures & Dimensional Art']
regex_pattern_wall = r"(?i)(\d*\.?\d*.?Inch\s?x\s?\d*\.?\d*.?Inch)|(\d*.?Inch)|(\d*\.\d*.?Inch)|()" 
wall['matches'] =wall['long_desc'].apply(lambda x: re_extract(regex_pattern_wall, x))
wall['name_matches'] =wall['product_name'].apply(lambda x: re_extract(regex_pattern_wall, x))
wall_df=wall[wall['name_matches'].astype(str)!='[]']

# Square

In [ ]:
wall_df['name_matches']=wall_df['name_matches'].str.get(0)
wall_df[['length', 'width']]=wall_df['name_matches'].str.split("x",expand=True,)
wall_df.head(1)

In [ ]:
wall_df['length']=wall_df['length'].apply(lambda measure: re.sub(r"\-Inch",'',str(measure)))
wall_df['width']=wall_df['width'].apply(lambda measure: re.sub(r"\-Inch",'',str(measure)))  
wall_df['length']=wall_df['length'].apply(lambda measure: re.sub(r" ",'',str(measure)))
wall_df['width']=wall_df['width'].apply(lambda measure: re.sub(r" ",'',str(measure))) 
wall_df['length']=wall_df['length'].astype(str)
wall_df['width']=wall_df['width'].astype(str)

In [ ]:
wall_df['compared']=wall_df['length'].equals(wall_df['width'])
wall_df.loc[(wall_df['length']==wall_df['width']), 'compared']=wall_df['length']
wall_new=wall_df[wall_df['compared'].astype(str)!='False']
wall_new[curation_col]=wall_new['compared']   
wall_new[curation_col]=wall_new[curation_col].apply(lambda x: x+' in')
wall_new.head(1)

In [ ]:
len(wall_new)

In [ ]:
square=wall_df[wall_df['width'].astype(str)=='None']
square[curation_col]=square['length']
square[curation_col]=square[curation_col].apply(lambda x: x+' in')
square.head(1)

In [ ]:
len(square)

# Trees

In [ ]:
trees=df[df['buckets'].astype(str)=='Trees, Shrubs, Topiaries, & Other Plants']
# \d*.?Foot
regex_pattern_tree = r"(?i)(\b\d*.?Foot\s?\d+.?Inch)|(\b\d*.?\d*\/\d*.?Foot)|(\b\d*\.?\d*.?Foot)|(\b\d*.?Foot)|()" 
trees['matches'] =trees['long_desc'].apply(lambda x: re_extract(regex_pattern_tree, x))
trees['name_matches'] =trees['product_name'].apply(lambda x: re_extract(regex_pattern_tree, x))
trees_df=trees[trees['name_matches'].astype(str)!='[]']

In [ ]:
#import regex as re
matches_tree=trees_df
matches_tree[curation_col]=matches_tree['name_matches'].astype(str)
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda x: f'"{x}"')
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"\-Foot",' ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'\-foot',' ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"\-1\/2",'.5',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r' 1\/2','.5',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r" Foot",' ft',str(measure)))

matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'7 ft 6\-Inch','7.5 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"6 ft 5\-Inch",'6.4 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'4 ft 6\-Inch','4.6 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"7 ft 5\-Inch",'7.4 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"2 ft 5\-Inch",'2.4 ft',str(measure)))

matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
matches_tree.head(10)

In [ ]:
matches_tree[curation_col].explode().value_counts()

In [ ]:
len(matches_tree)

# Stools

In [ ]:
stools=df[df['buckets'].astype(str)=='Stools']
regex_pattern_stool = r"(?i)(\d*\.\d*.?Inch)|(\d*.?Inch)|(\d.?.?H)|()" 
stools['matches'] =stools['long_desc'].apply(lambda x: re_extract(regex_pattern_stool, x))
stools['name_matches'] =stools['product_name'].apply(lambda x: re_extract(regex_pattern_stool, x))
stools_df=stools[stools['name_matches'].astype(str)!='[]']

In [ ]:
#import regex as re
matches_stool=stools_df
matches_stool[curation_col]=matches_stool['name_matches'].astype(str)
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda x: f'"{x}"')
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r"\-Inch",' in',str(measure)))
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r" Inch",' in',str(measure)))
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
matches_stool.head(10)

In [ ]:
matches_stool[curation_col].explode().value_counts()

In [ ]:
len(matches_stool)

# Side Tables

In [ ]:
side_tables=df[df['buckets'].astype(str)=='Side/End Tables']
regex_pattern_table = r"(?i)(\d*\.\d*”.?H)|(\d*\.\d*.?Inch)|(\d*.?Inch)|(\d*”.?H)|()" 
side_tables['matches'] =side_tables['long_desc'].apply(lambda x: re_extract(regex_pattern_table, x))
side_tables['name_matches'] =side_tables['product_name'].apply(lambda x: re_extract(regex_pattern_table, x))
side_tables_df=side_tables[side_tables['matches'].astype(str)!='[]']
side_tables_df['matches']= side_tables_df['matches'].apply(lambda x: sorted(x, key=natural_key))

In [ ]:
#import regex as re
matches_side_table=side_tables_df
matches_side_table[curation_col]=side_tables_df['matches'].astype(str)
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda x: f'"{x}"')
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r'\” H',' in',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r'\”H',' in',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r" ",'',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r"in",' in',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))

In [ ]:
len(matches_side_table)

# Sculptures and figurines

In [ ]:
sculptures=df[df['buckets'].astype(str)=='Sculptures, Figurines, & Decorative Objects']
# \d*.?Foot
regex_pattern_sc = r"(?i)(\b\d*.?Foot\s?\d+.?Inch)|(\b\d*.?\d*\/\d*.?Foot)|(\b\d*\.?\d*.?Foot)|(\b\d*.?Foot)|(\d*\.\d*”.?H)|(\d*\.\d*.?Inch.?H)|(\d*.?Inch.?H)|(\d*”.?H)|()" 
sculptures['matches'] =sculptures['long_desc'].apply(lambda x: re_extract(regex_pattern_sc, x))
sculptures['name_matches'] =sculptures['product_name'].apply(lambda x: re_extract(regex_pattern_sc, x))
sculptures_df=sculptures[(sculptures['matches'].astype(str)!='[]')]
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='69241383']
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='16623601']
sculptures_df['matches']= sculptures_df['matches'].apply(lambda x: sorted(x, key=natural_key))
# sculptures_df

In [ ]:
regex_pattern_sc_mis = r"(?i)(\b\d*.?Foot\s?\d+.?Inch)|(\b\d*.?\d*\/\d*.?Foot)|(\b\d*\.?\d*.?Foot)|(\b\d*.?Foot)|(\d*\.\d*''.?H)|(\d*\.\d*.?Inch.?H)|(\d*.?Inch.?H)|(\d*''.?H)|()" 
sculptures['mis_matches'] =sculptures['long_desc'].apply(lambda x: re_extract(regex_pattern_sc_mis, x))
sculptures['misc_name_matches'] =sculptures['product_name'].apply(lambda x: re_extract(regex_pattern_sc_mis, x))
sculptures_df=sculptures[(sculptures['misc_name_matches'].astype(str)!='[]')]
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='69241383']
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='16623601']
sculptures_df['matches']= sculptures_df['matches'].apply(lambda x: sorted(x, key=natural_key))
sculptures_df

In [ ]:
#import regex as re
matches_sc=sculptures_df
matches_sc[curation_col]=sculptures_df['matches'].astype(str)

matches_sc[curation_col]=matches_sc[curation_col].apply(lambda x: f'"{x}"')
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'\”','',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'H',' in',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'  ',' ',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'Inch H',' in',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'Inch','',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"\-",'',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"in",' in',str(measure)))

matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'1 in","16 in','16 in',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"6 ft 5\-Inch",'6.4 ft',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'4 ft 6\-Inch','4.6 ft',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"7 ft 5\-Inch",'7.4 ft',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"2 ft 5\-Inch",'2.4 ft',str(measure)))

matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches_sc.head(10)

In [ ]:
matches_sc[curation_col].explode().value_counts()

In [ ]:
matches_sc[matches_sc['matches'].astype(str)==' foot']

In [ ]:
len(matches_sc)

# foot

In [ ]:
# regex_pattern = r"(?i)(\b\d+\.\d*.?\"\s?H\s)|(\b\d+.?\"\s?H)|(\b\d+\.\d*.?\"\s?H)|(\d*\s?\-\d*\/\d*\\\"\s?H)|(\d*\s\-?\d*\/\d*\\\"\s?H)|()" 
# regex_pattern_diam = r'(?i)(\d+\.\d*\s?\\\"\s?i?n?\s?diam)|(\d*\s?\\\"\s?i?n?\s?diam)|()'
# regex_pattern_foot = r"(?i)(\b\d+\.\d*\'\s?H\s)|(\b\d+.?\'\s?H)|(\b\d+\.\d*\'\s?H)|(\d*\s?\-\d*\/\d*\\?\'\s?H)|(\d*\s\-?\d*\/\d*\\\'\s?H)|()" 

In [ ]:
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# df['diam_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diam, x))
# df['foot_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_foot, x))
# len(df)

In [ ]:
# foot=df[df['foot_matches'].astype(str)!='[]']
# foot=foot[foot['buckets'].astype(str)=='Trees, Shrubs, Topiaries, & Other Plants']
# len(foot)

In [ ]:
# # x=df[df['matches'].astype(str)!='[]']
# x=df
# len(x)
# # len(x)

In [ ]:
# x=x[x['matches'].astype(str)!='[]']

In [ ]:
# replacement_foot = {
#     "\' H": " ft",
#     "' H": "ft",
#     '  ft': " ft",
#     "\'H": " ft",
#     "' h": " ft",
#     "3\' H": "3 ft"
# }

In [ ]:
# # Replacement
# foot['foot_matches']= foot['foot_matches'].apply(
#     lambda x: [replace_all(string, replacement_foot) for string in x] if type(x)==list else x)

In [ ]:
# # Replacement
# foot['foot_matches']= foot['foot_matches'].apply(
#     lambda x: [replace_all(string, replacement_foot) for string in x] if type(x)==list else x)
# # foot.head(3)

In [ ]:
# foot['foot_matches']= foot['foot_matches'].apply(
#     lambda x: sorted(x, key=natural_key))
# # foot.head(3)

In [ ]:
# #import regex as re
# matches_foot=foot
# matches_foot[curation_col]=matches_foot['foot_matches'].astype(str)
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda x: f'"{x}"')
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches_foot.head(3)

In [ ]:
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r"\-1\/2",'.5',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'\"\\xa0H',' ft',str(measure)))

In [ ]:
# matches_foot[curation_col].explode().value_counts()

In [ ]:
replacement = {
    '\" H ': " in",
    "\ in": " in",
    '\" H': " in",
    '\" H': " in",
    '\" H ': " in",
    "\ in": " in",
    '\" H': " in",
    '\" H': " in",
    '\" h': " in",
    '\"H': " in"
}

In [ ]:
x=x[x['matches'].astype(str)!='[]']
x.head(1)

In [ ]:
# Replacement
x['matches']= x['matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

In [ ]:
# Replacement
x['matches']= x['matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)
x.head(1)

In [ ]:
x['matches']= x['matches'].apply(
    lambda x: sorted(x, key=natural_key))
x.head(1)

In [ ]:
# x=x[(x['matches'].astype(str)!='3/4 in')&(x['matches'].astype(str)!='1/4 in')&(x['matches'].astype(str)!='1/5 in')&(x['matches'].astype(str)!=' 3/4 in')]

In [ ]:
# replacement_diam = {
#     'diam': "",
#     " ": "",
#     '\"': "",
#     'in': "",
#     'Diam': ""
# }
# # Replacement
# diam['diam_matches']= diam['diam_matches'].apply(
#     lambda x: [replace_all(string, replacement_diam) for string in x] if type(x)==list else x)

# # Replacement
# diam['diam_matches']= diam['diam_matches'].apply(
#     lambda x: [replace_all(string, replacement_diam) for string in x] if type(x)==list else x)
# diam.head(3)

# diam['diam_matches']= diam['diam_matches'].apply(
#     lambda x: sorted(x, key=natural_key))
# diam.head(3)

# matches

In [ ]:
# matches=x

In [ ]:
# #import regex as re
# matches[curation_col]=matches['matches'].astype(str)
# matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches.head(3)

In [ ]:
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1\/2','.5',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3\/4','.75',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3\/4','.75',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1\/2','.5',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1\/4','.25',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1\/4','.25',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3\/8','.38',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3\/8','.38',str(measure)))

# matches=matches[matches[curation_col].astype(str)!='["3/4 in"]']
# matches=matches[matches[curation_col].astype(str)!='["1/4 in"]']
# matches=matches[matches[curation_col].astype(str)!='["1/5 in"]']

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-4\/5','.8',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 4\/5','.8',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/5','.2',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/5','.2',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-5/8','.62',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 5/8','.62',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-2/3','.66',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 2/3','.66',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3/10','.3',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3/10','.3',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/3','.33',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/3','.33',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-8/9','.89',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 8/9','.89',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3/5','.6',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3/5','.6',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-2/5','.4',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 2/5','.4',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-7/10','.7',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 7/10','.7',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/10','.1',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/10','.1',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/8','.13',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/8','.13',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-7/8','.88',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 7/8','.88',str(measure)))

In [ ]:
# # matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'1575 in','15.75 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\\xa0','',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'4563 in','45.63 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'3625 in','36.25 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\.625','.63',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\.375','.38',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\.875','.88',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'335 in","635 in','33.5 in","63.5 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'45 in","395 in','45 in","39.5 in',str(measure)))

In [ ]:
# matches[matches[curation_col].astype(str)=='["0.59 in"]']

In [ ]:
# matches[matches[curation_col].astype(str)=='["45 in","395 in"]']

In [ ]:
len(matches)

In [ ]:
# matches_diam=diam

In [ ]:
# #import regex as re
# matches_diam[curation_col]=matches_diam['diam_matches'].astype(str)
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda x: f'"{x}"')
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches_diam.head(3)

In [ ]:
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\\\\',' in',str(measure)))

In [ ]:
# matches_diam=matches_diam[matches_diam['buckets'].astype(str)!='Trees, Shrubs, Topiaries, & Other Plants']
# matches_diam=matches_diam[matches_diam['buckets'].astype(str)=='Hanging Accents & Accessories']

In [ ]:
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\\',' in',str(measure)))

In [ ]:
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'.875','.88',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\"10 in\"\,\"10 in\"','"10 in"',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'.725','.73',str(measure)))

In [ ]:
# matches_diam[curation_col].explode().value_counts()[0:500]

In [ ]:
# error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets-height-sculpture - {attribute}-- match upload_Shawn.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matches_sc)

In [ ]:
def looks_good_na(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets-NA - {attribute}-- match upload_Shawn.csv',index=False)

In [ ]:
looks_good_na('Bed Bath & Beyond', attribute, df, na_upload)

In [ ]:
matches_side_table+matches_stool+matches_tree